In [145]:
data = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

In [132]:
with open("AoC_D12_data.txt", "r") as f:
    data = f.read()

In [146]:
import pandas as pd
garden = pd.DataFrame([list(line) for line in data.split("\n")])
padded_garden = garden.copy()
padded_garden.loc[-1, :] = '.'
padded_garden.loc[len(garden.index), :] = '.'
padded_garden.loc[:, len(garden.columns)] = '.'
padded_garden.loc[:, -1] = '.'
padded_garden = padded_garden.sort_index()
padded_garden = padded_garden[sorted(padded_garden.columns)]
padded_garden

,-1,0,1,2,3,4,5,6,7,8,9,10
-1,.,.,.,.,.,.,.,.,.,.,.,.
0,.,R,R,R,R,I,I,C,C,F,F,.
1,.,R,R,R,R,I,I,C,C,C,F,.
2,.,V,V,R,R,R,C,C,F,F,F,.
3,.,V,V,R,C,C,C,J,F,F,F,.
4,.,V,V,V,V,C,J,J,C,F,E,.
5,.,V,V,I,V,C,C,J,J,E,E,.
6,.,V,V,I,I,I,C,J,J,E,E,.
7,.,M,I,I,I,I,I,J,J,E,E,.
8,.,M,I,I,I,S,I,J,E,E,E,.


In [147]:
def find_region(pos):
    region = {pos}
    veggy_type = garden.loc[pos[1], pos[0]]
    aera = 0
    fences = set()
    while len(region) > aera:
        temp_region = region.copy()
        aera = len(region)
        for pos in temp_region:
            if padded_garden.loc[pos[1]-1, pos[0]] == veggy_type:
                region.add((pos[0], pos[1]-1))
                IN_EXISTING_REGIONS.add((pos[0], pos[1]-1))
            else:
                fences.add((pos, (pos[0], pos[1]-1)))
                
            if padded_garden.loc[pos[1]+1, pos[0]] == veggy_type:
                region.add((pos[0], pos[1]+1))
                IN_EXISTING_REGIONS.add((pos[0], pos[1]+1))
            else:
                fences.add((pos, (pos[0], pos[1]+1)))
                
            if padded_garden.loc[pos[1], pos[0]-1] == veggy_type:
                region.add((pos[0]-1, pos[1]))
                IN_EXISTING_REGIONS.add((pos[0]-1, pos[1]))
            else:
                fences.add((pos, (pos[0]-1, pos[1])))
            
            if padded_garden.loc[pos[1], pos[0]+1] == veggy_type:
                region.add((pos[0]+1, pos[1]))
                IN_EXISTING_REGIONS.add((pos[0]+1, pos[1]))
            else:
                fences.add((pos, (pos[0]+1, pos[1])))
    return region, fences, len(fences), len(region)
            

In [148]:
IN_EXISTING_REGIONS = set()
regions = {}
region_id = 0
for x in garden.columns:
    for y in garden.index:
        pos = (x, y)
        if pos not in IN_EXISTING_REGIONS:
            regions[region_id] = {}
            regions[region_id]["cells"], regions[region_id]["fences"],  regions[region_id]["len_fences"], regions[region_id]["aera"] = find_region(pos)
            region_id +=1

In [149]:
modern_business_practices = 0
for region in regions.items():
    modern_business_practices += region[1]["len_fences"]*region[1]["aera"]
modern_business_practices

1930

In [150]:
# PART 2
def calculated_discounted_fences(fences):
    linear_fences = []
    already_in_a_linear_fence = set()
    for fence in fences:
        if fence not in already_in_a_linear_fence:
            if all([fence not in linear_fence for linear_fence in linear_fences]):
                linear_fence = {fence}
                already_in_a_linear_fence.add(fence)
                linear_fence_len = 0
                while linear_fence_len < len(linear_fence):
                    linear_fence_len = len(linear_fence)
                    for fence in fences:
                        if fence not in already_in_a_linear_fence:
                            inside, outside =fence[0], fence[1]
                            if any([((inside[0]+1, inside[1]), (outside[0]+1, outside[1]))== f for f in linear_fence]):
                                linear_fence.add(fence)
                                already_in_a_linear_fence.add(fence)
                            if any([((inside[0]-1, inside[1]), (outside[0]-1, outside[1]))== f for f in linear_fence]):
                                linear_fence.add(fence)
                                already_in_a_linear_fence.add(fence)
                            if any([((inside[0], inside[1]+1), (outside[0], outside[1]+1))== f for f in linear_fence]):
                                linear_fence.add(fence)
                                already_in_a_linear_fence.add(fence)
                            if any([((inside[0], inside[1]-1), (outside[0], outside[1]-1))== f for f in linear_fence]):
                                linear_fence.add(fence)
                                already_in_a_linear_fence.add(fence)
                # print(linear_fence)
                linear_fences.append(linear_fence)
    return linear_fences

In [151]:
modern_business_practices_discount = 0
for region in regions.items():
    modern_business_practices_discount += len(calculated_discounted_fences(region[1]["fences"])) * region[1]["aera"]
modern_business_practices_discount

1206